In [4]:
LANGUAGE = 'en'

FOR = 'for'
WHILE = 'while'
IF = 'if'
ELSE = 'else'
ELIF = 'elif'
RANGE = 'range'
IMPORT = 'import'
FROM = 'from'
WITH = 'with'
PRINT = 'print'
BREAK = 'break'
CONTINUE = 'continue'
ARITHEMATIC_OPERATORS = ['+', '-', '*', '/', '%', '**']
DEF = 'def'
CLASS = 'class'
SPACE = ' '
TAB = '\t'

class AlgoGenerator:
    """
    The Algorithm generator that handles the logic to convert a piece of code into
    it's corresponding algorithm
    """
    def get_value_between_brackets(self, line):
        i, j = 0, len(line) - 1
        bracket_start, bracket_end = i, j
        bracket_start_found, bracket_end_found = False, False
        while i < j:
            if bracket_start_found and bracket_end_found:
                break
            if not bracket_start_found and line[i] == '(':
                bracket_start = i
                bracket_start_found = True
            elif not bracket_end_found and line[j] == ')':
                bracket_end = j
                bracket_end_found = True
            if not bracket_start_found:
                i += 1
            if not bracket_end_found:
                j -= 1

        return line[bracket_start + 1:bracket_end]


    def handle_loop(self, loop_type, line):
        line = line.lstrip()
        start = 0
        if loop_type == FOR:
            if RANGE in line:
                iterator = line.split()[1]
                value = self.get_value_between_brackets(line)
                if "," in value:
                    start = int(value.split(",")[0])
                    iterations = int(value.split(",")[1]) - start
                else:
                    iterations = value

                return "Start a for-loop {} times with iterator '{}' starting from {}".format(iterations, iterator, start)

            return "Start a for-loop on the iterable '{}' with iterator(s) '{}'".format(line.split("in")[1][:-1].strip(), line.split("in")[0].replace("for", "").strip())

        if loop_type == WHILE:
            condition = line[5:-1]
            return "Run a while-loop with condition {}".format(condition)


    def handle_assignment(self, line):
        if line[line.index("=") - 1] in ARITHEMATIC_OPERATORS:
            operator = line[line.index("=") - 1]
            operand = line[:line.index("=") - 1].strip()
            if operator == '+':
                operation = "Increment"
            elif operator == '-':
                operation = "Decrement"
            elif operator == '/':
                operation = "Divide"
            elif operator == '*' and line[line.index("=") - 2] == '*':
                operation = "Raise"
                operand = line[:line.index("=") - 2].strip()
            elif operator == '*':
                operation = "Multiply"
            elif operator == '%':
                operation = "Modulo"

            return "{} the value of '{}' by {}".format(operation, operand, line.split("=")[1].strip())
        return "Set the value of '{}' to '{}'".format(line.split('=')[0].strip(), line.split('=')[1].strip())


    def get_the_value_to_print(self, line):
        value = self.get_value_between_brackets(line)
        if value[0] == "'" or value[0] == '"':
            return "Print {}".format(value)

        return "Print the value of '{}'".format(value)


    def handle_function_definition(self, line):
        function_name = line.lstrip().split("(")[0][4:]
        params = self.get_value_between_brackets(line)

        return "Define a function '{}' with parameter(s) '{}'".format(function_name, params)


    def handle_class_declaration(self, line):
        line = line.lstrip()
        if '(' in line:
            class_name = line.split("(")[0][6:]
            base_class_names = self.get_value_between_brackets(line)

            return "Define a class '{}' that inherits from '{}'".format(class_name, base_class_names)

        class_name = line[6:-1]

        return "Define a class '{}'".format(class_name)


    def get_keyword(self, line):
        keyword = None
        line = line.lstrip()

        if line[:3] == FOR and line[3] == " ":
            keyword = FOR
        elif line[:5] == WHILE and line[5] == " ":
            keyword = WHILE
        elif line[:2] == IF and line[2] == " ":
            keyword = IF
        elif line[:4] == ELIF and line[4] == " ":
            keyword = ELIF
        elif line[:4] == ELSE and line[4] == ":":
            keyword = ELSE
        elif line[:4] == FROM and line[4] == " ":
            keyword = FROM
        elif line[:6] == IMPORT and line[6] == " ":
            keyword = IMPORT
        elif line[:4] == WITH and line[4] == " ":
            keyword = WITH
        elif line[:5] == PRINT and line[5] == "(":
            keyword = PRINT
        elif line[:8] == CONTINUE:
            keyword = CONTINUE
        elif line[:5] == BREAK:
            keyword = BREAK
        elif line[:3] == DEF and line[3] == " ":
            keyword = DEF
        elif line[:5] == CLASS and line[5] == " ":
            keyword = CLASS

        return keyword

In [11]:
import os
import traceback
import sys
from gtts import gTTS
# from generator_engine import AlgoGenerator
# from constants import (LANGUAGE, FOR, WHILE, IF, ELSE, ELIF, IMPORT, FROM, PRINT, BREAK, CONTINUE, 
#                        DEF, SPACE, TAB, CLASS9)
# from configurations import SET_LINE_NUMBER, EXPORT_TO_FILE, GENERATE_AUDIO

SET_LINE_NUMBER = False
EXPORT_TO_FILE = True
GENERATE_AUDIO = False

source_filename = "car.py"
generator = AlgoGenerator()
algorithm = ""
line_number = 1

with open(source_filename) as f:
    for line in f:
        if SET_LINE_NUMBER:
            algorithm += str(line_number) + ' '

        line = line.rstrip()
        if not line:
            algorithm += "\n"  # If we find a blank line, we keep the blank line
        else:
            try:
                indentation_type = SPACE if line[0] == SPACE else TAB
                indentation_value = len(line) - len(line.lstrip())
                algorithm += indentation_type * indentation_value

                keyword = generator.get_keyword(line)

                if keyword in [FOR, WHILE]:
                    algorithm += generator.handle_loop(keyword, line)
                elif keyword in [IF, ELSE, ELIF]:
                    algorithm += line.strip().capitalize()
                elif keyword in [IMPORT, FROM]:
                    algorithm += line.strip().capitalize()
                elif keyword == PRINT:
                    algorithm += generator.get_the_value_to_print(line)
                elif keyword in [BREAK, CONTINUE]:
                    algorithm += line.strip().capitalize()
                elif keyword == DEF:
                    algorithm += generator.handle_function_definition(line)
                elif keyword == CLASS:
                    algorithm += generator.handle_class_declaration(line)
                elif "==" not in line and "=" in line:
                    algorithm += generator.handle_assignment(line)
                else:
                    algorithm += line.strip()

                algorithm += "\n"

            except Exception:
                traceback.print_exc()
                print("Could not translate line number {}-{}".format(line_number, line))

        line_number += 1

if EXPORT_TO_FILE:
    with open('algo_output.txt', 'w') as f_out:
        f_out.write(algorithm)
else:
    print(algorithm)

if GENERATE_AUDIO:
    myobj = gTTS(text=algorithm, lang=LANGUAGE, slow=False)
    myobj.save("algo_generated.mp3")
    os.system("mpg321 algo_generated.mp3")